In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def firsts(list):
    answer=[]
    for pair in list:
        answer=answer+[pair[0]]
    return answer

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def last_element(listo):return listo[len(listo)-1]
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [ ]:
#output snipped; I interrupted this after k = 94.
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [1..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-nszfunction(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
#below was originally done in a separate cell
#import pickle
#wfile = open('/Users/barrybrent/26oct22no1.txt','wb')
#t = pickle.dumps(str(polydata))
#wfile.write(t)
#wfile.close()

In [14]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

for n in [1..100]:
    p=prime(n)
    firsts=[]
    for k in [0..93]:
        m2=[]
        data=[]
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [3..1003]: # nb
            m=p^j+p^(j-1)
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            if mod(o,2)==1:m2=m2+[mod(power,2)]
        m2=uniteList(m2)
        print((p,power,m2))

(2, 1, [1])
(2, 2, [])
(2, 3, [1])
(2, 4, [])
(2, 5, [1])
(2, 6, [])
(2, 7, [1])
(2, 8, [])
(2, 9, [1])
(2, 10, [])
(2, 11, [1])
(2, 12, [])
(2, 13, [1])
(2, 14, [])
(2, 15, [1])
(2, 16, [])
(2, 17, [1])
(2, 18, [])
(2, 19, [1])
(2, 20, [])
(2, 21, [1])
(2, 22, [])
(2, 23, [1])
(2, 24, [])
(2, 25, [1])
(2, 26, [])
(2, 27, [1])
(2, 28, [])
(2, 29, [1])
(2, 30, [])
(2, 31, [1])
(2, 32, [])
(2, 33, [1])
(2, 34, [])
(2, 35, [1])
(2, 36, [])
(2, 37, [1])
(2, 38, [])
(2, 39, [1])
(2, 40, [])
(2, 41, [1])
(2, 42, [])
(2, 43, [1])
(2, 44, [])
(2, 45, [1])
(2, 46, [])
(2, 47, [1])
(2, 48, [])
(2, 49, [1])
(2, 50, [])
(2, 51, [1])
(2, 52, [])
(2, 53, [1])
(2, 54, [])
(2, 55, [1])
(2, 56, [])
(2, 57, [1])
(2, 58, [])


KeyboardInterrupt: 

In [6]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
no2=[]
no3=[]
for n in [1..50]:
    p=prime(n)
    firsts=[]
    for k in [0..50]: 
        data=[]
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..50]: # nb
            m=prime(j)
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            if o!=0:print((p,power,m,o))

(2, 1, 2, 5)
(2, 2, 2, 12)
(2, 3, 2, 17)
(2, 4, 2, 26)
(2, 5, 2, 31)
(2, 6, 2, 38)
(2, 7, 2, 43)
(2, 8, 2, 54)
(2, 9, 2, 59)
(2, 10, 2, 66)
(2, 11, 2, 71)
(2, 12, 2, 80)
(2, 13, 2, 85)
(2, 14, 2, 92)
(2, 15, 2, 97)
(2, 16, 2, 110)
(2, 17, 2, 115)
(2, 18, 2, 122)
(2, 19, 2, 127)
(2, 20, 2, 136)
(2, 21, 2, 141)
(2, 22, 2, 148)
(2, 23, 2, 153)
(2, 24, 2, 164)
(2, 25, 2, 169)
(2, 26, 2, 176)
(2, 27, 2, 181)
(2, 28, 2, 190)
(2, 29, 2, 195)
(2, 30, 2, 202)
(2, 31, 2, 207)
(2, 32, 2, 222)
(2, 33, 2, 227)
(2, 34, 2, 234)
(2, 35, 2, 239)
(2, 36, 2, 248)
(2, 37, 2, 253)
(2, 38, 2, 260)
(2, 39, 2, 265)
(2, 40, 2, 276)
(2, 41, 2, 281)
(2, 42, 2, 288)
(2, 43, 2, 293)
(2, 44, 2, 302)
(2, 45, 2, 307)
(2, 46, 2, 314)
(2, 47, 2, 319)
(2, 48, 2, 332)
(2, 49, 2, 337)
(2, 50, 2, 344)
(2, 51, 2, 349)
(3, 1, 2, -1)
(3, 1, 5, -1)
(3, 1, 7, -1)
(3, 1, 11, -1)
(3, 1, 13, -1)
(3, 1, 17, -1)
(3, 1, 19, -1)
(3, 1, 23, -1)
(3, 1, 29, -1)
(3, 1, 31, -1)
(3, 1, 37, -1)
(3, 1, 41, -1)
(3, 1, 43, -1)
(3, 1, 47, -1)
(3

(3, 17, 41, 3)
(3, 17, 43, 1)
(3, 17, 47, 2)
(3, 17, 53, 1)
(3, 17, 59, 3)
(3, 17, 61, 2)
(3, 17, 67, 3)
(3, 17, 71, 2)
(3, 17, 73, 2)
(3, 17, 79, 2)
(3, 17, 83, 2)
(3, 17, 89, 2)
(3, 17, 97, 1)
(3, 17, 101, 2)
(3, 17, 103, 3)
(3, 17, 107, 1)
(3, 17, 109, 1)
(3, 17, 113, 3)
(3, 17, 127, 2)
(3, 17, 131, 2)
(3, 17, 137, 3)
(3, 17, 139, 2)
(3, 17, 149, 3)
(3, 17, 151, 1)
(3, 17, 157, 3)
(3, 17, 163, 1)
(3, 17, 167, 3)
(3, 17, 173, 1)
(3, 17, 179, 3)
(3, 17, 181, 4)
(3, 17, 191, 2)
(3, 17, 193, 2)
(3, 17, 197, 2)
(3, 17, 199, 2)
(3, 17, 211, 3)
(3, 17, 223, 2)
(3, 17, 227, 1)
(3, 17, 229, 3)
(3, 18, 3, 1)
(3, 18, 23, 1)
(3, 18, 59, 1)
(3, 18, 71, 1)
(3, 18, 73, 2)
(3, 18, 89, 1)
(3, 18, 97, 2)
(3, 18, 103, 1)
(3, 18, 137, 1)
(3, 18, 139, 3)
(3, 18, 227, 1)
(3, 19, 3, 2)
(3, 19, 23, 1)
(3, 19, 59, 1)
(3, 19, 71, 1)
(3, 19, 73, 2)
(3, 19, 89, 1)
(3, 19, 97, 2)
(3, 19, 103, 1)
(3, 19, 137, 1)
(3, 19, 139, 3)
(3, 19, 227, 1)
(3, 20, 2, 1)
(3, 20, 3, 3)
(3, 20, 5, 1)
(3, 20, 7, 1)
(3, 20, 11, 1

(3, 32, 197, 1)
(3, 32, 199, 1)
(3, 32, 223, 3)
(3, 32, 227, 2)
(3, 33, 3, 2)
(3, 33, 7, 1)
(3, 33, 13, 1)
(3, 33, 23, 1)
(3, 33, 41, 2)
(3, 33, 47, 5)
(3, 33, 53, 5)
(3, 33, 59, 1)
(3, 33, 61, 2)
(3, 33, 67, 2)
(3, 33, 71, 3)
(3, 33, 73, 2)
(3, 33, 89, 4)
(3, 33, 97, 1)
(3, 33, 101, 2)
(3, 33, 103, 1)
(3, 33, 107, 1)
(3, 33, 109, 1)
(3, 33, 137, 1)
(3, 33, 139, 1)
(3, 33, 149, 1)
(3, 33, 223, 2)
(3, 33, 227, 1)
(3, 33, 229, 4)
(3, 34, 3, 3)
(3, 34, 7, 1)
(3, 34, 13, 1)
(3, 34, 23, 1)
(3, 34, 41, 2)
(3, 34, 47, 5)
(3, 34, 53, 5)
(3, 34, 59, 1)
(3, 34, 61, 2)
(3, 34, 67, 2)
(3, 34, 71, 3)
(3, 34, 73, 2)
(3, 34, 89, 5)
(3, 34, 97, 1)
(3, 34, 101, 2)
(3, 34, 103, 1)
(3, 34, 107, 1)
(3, 34, 109, 1)
(3, 34, 137, 1)
(3, 34, 139, 1)
(3, 34, 149, 1)
(3, 34, 223, 2)
(3, 34, 227, 1)
(3, 34, 229, 2)
(3, 35, 2, 1)
(3, 35, 3, 4)
(3, 35, 5, 2)
(3, 35, 7, 2)
(3, 35, 11, 1)
(3, 35, 13, 3)
(3, 35, 17, 1)
(3, 35, 19, 1)
(3, 35, 23, 5)
(3, 35, 29, 1)
(3, 35, 31, 3)
(3, 35, 37, 1)
(3, 35, 41, 6)
(3, 35, 4

(3, 47, 131, 2)
(3, 47, 137, 2)
(3, 47, 139, 2)
(3, 47, 149, 2)
(3, 47, 151, 1)
(3, 47, 157, 1)
(3, 47, 163, 2)
(3, 47, 167, 1)
(3, 47, 173, 1)
(3, 47, 179, 3)
(3, 47, 181, 2)
(3, 47, 191, 1)
(3, 47, 193, 3)
(3, 47, 197, 6)
(3, 47, 199, 5)
(3, 47, 211, 4)
(3, 47, 223, 1)
(3, 47, 227, 2)
(3, 47, 229, 3)
(3, 48, 2, 1)
(3, 48, 3, 3)
(3, 48, 7, 1)
(3, 48, 13, 2)
(3, 48, 17, 2)
(3, 48, 19, 4)
(3, 48, 23, 1)
(3, 48, 31, 1)
(3, 48, 37, 4)
(3, 48, 43, 1)
(3, 48, 47, 1)
(3, 48, 53, 3)
(3, 48, 59, 1)
(3, 48, 67, 1)
(3, 48, 71, 1)
(3, 48, 73, 1)
(3, 48, 79, 2)
(3, 48, 83, 1)
(3, 48, 89, 1)
(3, 48, 97, 1)
(3, 48, 103, 1)
(3, 48, 107, 1)
(3, 48, 109, 1)
(3, 48, 113, 1)
(3, 48, 127, 3)
(3, 48, 131, 1)
(3, 48, 137, 1)
(3, 48, 139, 1)
(3, 48, 149, 1)
(3, 48, 179, 2)
(3, 48, 181, 2)
(3, 48, 193, 2)
(3, 48, 197, 2)
(3, 48, 199, 2)
(3, 48, 211, 3)
(3, 48, 227, 1)
(3, 48, 229, 2)
(3, 49, 2, 1)
(3, 49, 3, 4)
(3, 49, 7, 1)
(3, 49, 13, 2)
(3, 49, 17, 2)
(3, 49, 19, 4)
(3, 49, 23, 1)
(3, 49, 31, 1)
(3, 49, 37

(5, 14, 31, 2)
(5, 14, 37, 1)
(5, 14, 41, 2)
(5, 14, 43, 2)
(5, 14, 47, 1)
(5, 14, 53, 1)
(5, 14, 59, 2)
(5, 14, 61, 2)
(5, 14, 67, 2)
(5, 14, 71, 2)
(5, 14, 73, 3)
(5, 14, 79, 2)
(5, 14, 83, 2)
(5, 14, 89, 2)
(5, 14, 97, 1)
(5, 14, 101, 2)
(5, 14, 103, 1)
(5, 14, 107, 2)
(5, 14, 109, 2)
(5, 14, 113, 1)
(5, 14, 127, 2)
(5, 14, 131, 2)
(5, 14, 137, 1)
(5, 14, 139, 2)
(5, 14, 149, 2)
(5, 14, 151, 2)
(5, 14, 157, 3)
(5, 14, 163, 1)
(5, 14, 167, 2)
(5, 14, 173, 2)
(5, 14, 179, 2)
(5, 14, 181, 2)
(5, 14, 191, 2)
(5, 14, 193, 2)
(5, 14, 197, 1)
(5, 14, 199, 2)
(5, 14, 211, 2)
(5, 14, 223, 2)
(5, 14, 227, 2)
(5, 14, 229, 2)
(5, 15, 2, 1)
(5, 15, 3, 1)
(5, 15, 5, 9)
(5, 15, 7, 1)
(5, 15, 11, 1)
(5, 15, 17, 1)
(5, 15, 19, 1)
(5, 15, 23, 1)
(5, 15, 29, 1)
(5, 15, 31, 1)
(5, 15, 41, 1)
(5, 15, 43, 1)
(5, 15, 47, 2)
(5, 15, 53, 1)
(5, 15, 59, 1)
(5, 15, 61, 1)
(5, 15, 67, 1)
(5, 15, 71, 1)
(5, 15, 73, 2)
(5, 15, 79, 1)
(5, 15, 83, 1)
(5, 15, 89, 1)
(5, 15, 97, 1)
(5, 15, 103, 1)
(5, 15, 107, 1)
(5

...

(19, 18, 83, 1)
(19, 18, 97, 1)
(19, 18, 107, 1)
(19, 18, 131, 1)
(19, 18, 173, 1)
(19, 18, 197, 1)
(19, 18, 211, 1)
(19, 19, 19, 17)
(19, 19, 37, 1)
(19, 19, 41, 1)
(19, 19, 43, 1)
(19, 19, 79, 2)
(19, 19, 163, 1)
(19, 19, 223, 1)
(19, 20, 13, 1)
(19, 20, 19, 18)
(19, 20, 23, 2)
(19, 20, 167, 1)
(19, 21, 19, 19)
(19, 22, 19, 20)
(19, 23, 2, -1)
(19, 23, 3, -1)
(19, 23, 5, -1)
(19, 23, 7, -1)
(19, 23, 11, -1)
(19, 23, 13, -1)
(19, 23, 17, -1)
(19, 23, 19, 20)
(19, 23, 23, -1)
(19, 23, 29, -1)
(19, 23, 31, -1)
(19, 23, 37, -1)
(19, 23, 41, -1)
(19, 23, 43, -1)
(19, 23, 47, -1)
(19, 23, 53, -1)
(19, 23, 59, -1)
(19, 23, 61, -1)
(19, 23, 67, -1)
(19, 23, 71, -1)
(19, 23, 73, -1)
(19, 23, 79, -1)
(19, 23, 83, -1)
(19, 23, 89, -1)
(19, 23, 97, -1)
(19, 23, 101, -1)
(19, 23, 103, -1)
(19, 23, 107, -1)
(19, 23, 109, -1)
(19, 23, 127, -1)
(19, 23, 131, -1)
(19, 23, 137, -1)
(19, 23, 139, -1)
(19, 23, 149, -1)
(19, 23, 151, -1)
(19, 23, 157, -1)
(19, 23, 163, -1)
(19, 23, 167, -1)
(19, 23, 173,

(23, 17, 191, 1)
(23, 17, 223, 1)
(23, 17, 229, 1)
(23, 18, 2, 1)
(23, 18, 7, 2)
(23, 18, 23, 18)
(23, 18, 53, 1)
(23, 18, 67, 1)
(23, 18, 71, 1)
(23, 18, 113, 1)
(23, 18, 131, 1)
(23, 18, 163, 1)
(23, 18, 191, 1)
(23, 18, 223, 1)
(23, 19, 2, 1)
(23, 19, 7, 1)
(23, 19, 13, 1)
(23, 19, 23, 19)
(23, 19, 53, 1)
(23, 19, 59, 1)
(23, 19, 67, 1)
(23, 19, 71, 1)
(23, 19, 79, 1)
(23, 19, 113, 1)
(23, 19, 131, 1)
(23, 19, 151, 1)
(23, 19, 163, 1)
(23, 19, 191, 1)
(23, 19, 197, 1)
(23, 19, 223, 1)
(23, 20, 2, 1)
(23, 20, 7, 1)
(23, 20, 23, 20)
(23, 20, 53, 1)
(23, 20, 67, 1)
(23, 20, 71, 1)
(23, 20, 113, 1)
(23, 20, 131, 1)
(23, 20, 163, 1)
(23, 20, 191, 1)
(23, 20, 223, 1)
(23, 21, 2, 1)
(23, 21, 7, 1)
(23, 21, 23, 21)
(23, 21, 53, 1)
(23, 21, 67, 1)
(23, 21, 71, 1)
(23, 21, 113, 1)
(23, 21, 131, 1)
(23, 21, 163, 1)
(23, 21, 191, 1)
(23, 21, 223, 1)
(23, 22, 2, 1)
(23, 22, 7, 1)
(23, 22, 23, 22)
(23, 22, 53, 1)
(23, 22, 67, 1)
(23, 22, 71, 1)
(23, 22, 113, 1)
(23, 22, 131, 1)
(23, 22, 163, 1)
(

(29, 20, 67, 1)
(29, 20, 89, 1)
(29, 20, 107, 1)
(29, 20, 223, 1)
(29, 21, 2, 1)
(29, 21, 29, 21)
(29, 21, 31, 1)
(29, 21, 67, 1)
(29, 21, 89, 1)
(29, 21, 107, 1)
(29, 21, 223, 1)
(29, 22, 3, -1)
(29, 22, 5, -1)
(29, 22, 7, -1)
(29, 22, 11, -1)
(29, 22, 13, -1)
(29, 22, 17, -1)
(29, 22, 19, -1)
(29, 22, 23, -1)
(29, 22, 29, 21)
(29, 22, 37, -1)
(29, 22, 41, -1)
(29, 22, 43, -1)
(29, 22, 47, -1)
(29, 22, 53, -1)
(29, 22, 59, -1)
(29, 22, 61, -1)
(29, 22, 71, -1)
(29, 22, 73, -1)
(29, 22, 79, -1)
(29, 22, 83, -1)
(29, 22, 97, -1)
(29, 22, 101, -1)
(29, 22, 103, -1)
(29, 22, 109, -1)
(29, 22, 113, -1)
(29, 22, 127, -1)
(29, 22, 131, -1)
(29, 22, 137, -1)
(29, 22, 139, -1)
(29, 22, 149, -1)
(29, 22, 151, -1)
(29, 22, 157, -1)
(29, 22, 163, -1)
(29, 22, 167, -1)
(29, 22, 173, -1)
(29, 22, 179, -1)
(29, 22, 181, -1)
(29, 22, 191, -1)
(29, 22, 193, -1)
(29, 22, 197, -1)
(29, 22, 199, -1)
(29, 22, 211, -1)
(29, 22, 227, -1)
(29, 22, 229, -1)
(29, 23, 2, 1)
(29, 23, 29, 23)
(29, 23, 31, 1)
(29,

(31, 34, 103, -1)
(31, 34, 107, -1)
(31, 34, 109, -1)
(31, 34, 113, -1)
(31, 34, 127, -1)
(31, 34, 131, -1)
(31, 34, 137, -1)
(31, 34, 139, -1)
(31, 34, 149, -1)
(31, 34, 151, -1)
(31, 34, 157, -1)
(31, 34, 163, -1)
(31, 34, 167, -1)
(31, 34, 173, -1)
(31, 34, 179, -1)
(31, 34, 181, -1)
(31, 34, 191, -1)
(31, 34, 193, -1)
(31, 34, 197, -1)
(31, 34, 199, -1)
(31, 34, 211, -1)
(31, 34, 223, -1)
(31, 34, 227, -1)
(31, 34, 229, -1)
(31, 35, 31, 33)
(31, 36, 31, 34)
(31, 36, 89, 1)
(31, 36, 137, 1)
(31, 36, 151, 1)
(31, 36, 223, 1)
(31, 37, 5, 1)
(31, 37, 31, 35)
(31, 37, 53, 1)
(31, 37, 127, 1)
(31, 37, 211, 1)
(31, 38, 31, 36)
(31, 39, 31, 37)
(31, 40, 31, 38)
(31, 40, 47, 1)
(31, 40, 191, 1)
(31, 41, 31, 39)
(31, 41, 97, 1)
(31, 42, 31, 40)
(31, 42, 59, 1)
(31, 42, 131, 1)
(31, 42, 191, 1)
(31, 43, 7, 2)
(31, 43, 31, 41)
(31, 44, 31, 42)
(31, 45, 31, 43)
(31, 45, 211, 1)
(31, 46, 11, 1)
(31, 46, 31, 44)
(31, 46, 73, 1)
(31, 46, 113, 1)
(31, 46, 197, 1)
(31, 47, 2, 1)
(31, 47, 11, 1)
(31,

KeyboardInterrupt: 

In [56]:
prime(24)

89

In [7]:
prime(1000)

7919

In [80]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
no2=[]
no3=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        data=[]
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1.1]: # nb
            m=p
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            print((p,o,o-(p-2)))

(5, 3, 0)
(7, 5, 0)
(11, 9, 0)
(13, 11, 0)
(17, 15, 0)
(19, 17, 0)
(23, 21, 0)
(29, 27, 0)
(31, 29, 0)
(37, 35, 0)
(41, 39, 0)
(43, 41, 0)
(47, 45, 0)
(53, 51, 0)
(59, 57, 0)
(61, 59, 0)
(67, 65, 0)
(71, 69, 0)
(73, 71, 0)
(79, 77, 0)
(83, 81, 0)
(89, 87, 0)


In [81]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [2..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]: # nb
            m=p^2
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,m,o)
            data=data+[o]
            print(tupl)
print(data) # adjoining n = 2, p =3 to the cell below, there is no OEIS listing.

(3, 3, 9, 3)
(5, 5, 25, 8)
(7, 7, 49, 12)
(11, 11, 121, 20)
(13, 13, 169, 24)
(17, 17, 289, 32)
(19, 19, 361, 36)
(23, 23, 529, 44)
(29, 29, 841, 56)
(31, 31, 961, 60)
(37, 37, 1369, 72)
(41, 41, 1681, 80)
(43, 43, 1849, 84)
(47, 47, 2209, 92)
(53, 53, 2809, 104)
(59, 59, 3481, 116)
(61, 61, 3721, 120)
(67, 67, 4489, 132)
(71, 71, 5041, 140)
(73, 73, 5329, 144)
(79, 79, 6241, 156)
(83, 83, 6889, 164)
(89, 89, 7921, 176)
[3, 8, 12, 20, 24, 32, 36, 44, 56, 60, 72, 80, 84, 92, 104, 116, 120, 132, 140, 144, 156, 164, 176]


In [58]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]: # nb
            m=p^2
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,m,o)
            data=data+[o]
            print(tupl)
print(data) # A037168
# a(n) = 2*prime(n)-2, n = 3, 4, ....

(5, 5, 25, 8)
(7, 7, 49, 12)
(11, 11, 121, 20)
(13, 13, 169, 24)
(17, 17, 289, 32)
(19, 19, 361, 36)
(23, 23, 529, 44)
(29, 29, 841, 56)
(31, 31, 961, 60)
(37, 37, 1369, 72)
(41, 41, 1681, 80)
(43, 43, 1849, 84)
(47, 47, 2209, 92)
(53, 53, 2809, 104)
(59, 59, 3481, 116)
(61, 61, 3721, 120)
(67, 67, 4489, 132)
(71, 71, 5041, 140)
(73, 73, 5329, 144)
(79, 79, 6241, 156)
(83, 83, 6889, 164)
(89, 89, 7921, 176)
[8, 12, 20, 24, 32, 36, 44, 56, 60, 72, 80, 84, 92, 104, 116, 120, 132, 140, 144, 156, 164, 176]


In [60]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]: # nb
            m=p^3
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,m,o)
            data=data+[o]
            print(tupl)
print(data) # A176003, n = 4, 5, ...
# a(n) = 3*(n-th noncomposite number) - 2.

(5, 5, 125, 13)
(7, 7, 343, 19)
(11, 11, 1331, 31)
(13, 13, 2197, 37)
(17, 17, 4913, 49)
(19, 19, 6859, 55)
(23, 23, 12167, 67)
(29, 29, 24389, 85)
(31, 31, 29791, 91)
(37, 37, 50653, 109)
(41, 41, 68921, 121)
(43, 43, 79507, 127)
(47, 47, 103823, 139)
(53, 53, 148877, 157)
(59, 59, 205379, 175)
(61, 61, 226981, 181)
(67, 67, 300763, 199)
(71, 71, 357911, 211)
(73, 73, 389017, 217)
(79, 79, 493039, 235)
(83, 83, 571787, 247)
(89, 89, 704969, 265)
[13, 19, 31, 37, 49, 55, 67, 85, 91, 109, 121, 127, 139, 157, 175, 181, 199, 211, 217, 235, 247, 265]


In [61]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]: # nb
            m=p^4
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,m,o)
            data=data+[o]
            print(tupl)
print(data) # no OEIS listing

(5, 5, 625, 18)
(7, 7, 2401, 26)
(11, 11, 14641, 42)
(13, 13, 28561, 50)
(17, 17, 83521, 66)
(19, 19, 130321, 74)
(23, 23, 279841, 90)
(29, 29, 707281, 114)
(31, 31, 923521, 122)
(37, 37, 1874161, 146)
(41, 41, 2825761, 162)
(43, 43, 3418801, 170)
(47, 47, 4879681, 186)
(53, 53, 7890481, 210)
(59, 59, 12117361, 234)
(61, 61, 13845841, 242)
(67, 67, 20151121, 266)
(71, 71, 25411681, 282)
(73, 73, 28398241, 290)
(79, 79, 38950081, 314)
(83, 83, 47458321, 330)
(89, 89, 62742241, 354)
[18, 26, 42, 50, 66, 74, 90, 114, 122, 146, 162, 170, 186, 210, 234, 242, 266, 282, 290, 314, 330, 354]


In [62]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]: # nb
            m=p^5
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,m,o)
            data=data+[o]
            print(tupl)
print(data) # no listing

(5, 5, 3125, 23)
(7, 7, 16807, 33)
(11, 11, 161051, 53)
(13, 13, 371293, 63)
(17, 17, 1419857, 83)
(19, 19, 2476099, 93)
(23, 23, 6436343, 113)
(29, 29, 20511149, 143)
(31, 31, 28629151, 153)
(37, 37, 69343957, 183)
(41, 41, 115856201, 203)
(43, 43, 147008443, 213)
(47, 47, 229345007, 233)
(53, 53, 418195493, 263)
(59, 59, 714924299, 293)
(61, 61, 844596301, 303)
(67, 67, 1350125107, 333)
(71, 71, 1804229351, 353)
(73, 73, 2073071593, 363)
(79, 79, 3077056399, 393)
(83, 83, 3939040643, 413)
(89, 89, 5584059449, 443)
[23, 33, 53, 63, 83, 93, 113, 143, 153, 183, 203, 213, 233, 263, 293, 303, 333, 353, 363, 393, 413, 443]


In [64]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]: # nb
            m=p^p
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,o)
            data=data+[o]
            print(tupl)
print(data) # A049001, n = 3, 4, ...; 	a(n) = prime(n)^2 - 2.

(5, 5, 23)
(7, 7, 47)
(11, 11, 119)
(13, 13, 167)
(17, 17, 287)
(19, 19, 359)
(23, 23, 527)
(29, 29, 839)
(31, 31, 959)
(37, 37, 1367)
(41, 41, 1679)
(43, 43, 1847)
(47, 47, 2207)
(53, 53, 2807)
(59, 59, 3479)
(61, 61, 3719)
(67, 67, 4487)
(71, 71, 5039)
(73, 73, 5327)
(79, 79, 6239)
(83, 83, 6887)
(89, 89, 7919)
[23, 47, 119, 167, 287, 359, 527, 839, 959, 1367, 1679, 1847, 2207, 2807, 3479, 3719, 4487, 5039, 5327, 6239, 6887, 7919]


In [65]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]: # nb
            m=p^((p-1)/2)
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,o)
            data=data+[o]
            print(tupl)
print(data) # no OEIS entry

(5, 5, 8)
(7, 7, 19)
(11, 11, 53)
(13, 13, 76)
(17, 17, 134)
(19, 19, 169)
(23, 23, 251)
(29, 29, 404)
(31, 31, 463)
(37, 37, 664)
(41, 41, 818)
(43, 43, 901)
(47, 47, 1079)
(53, 53, 1376)
(59, 59, 1709)
(61, 61, 1828)
(67, 67, 2209)
(71, 71, 2483)
(73, 73, 2626)
(79, 79, 3079)
(83, 83, 3401)
(89, 89, 3914)
[8, 19, 53, 76, 134, 169, 251, 404, 463, 664, 818, 901, 1079, 1376, 1709, 1828, 2209, 2483, 2626, 3079, 3401, 3914]


In [66]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]: # nb
            if mod(p,3)==1:
                m=p^((p-1)/3)
                polydm=evaluate(polyd,m)
                o=order(p,polydm)
                tupl=(p,power,m,o/2)
                data=data+[o/2]
                print(tupl)
print(data) # no OEIS entry

(7, 7, 49, 6)
(13, 13, 28561, 25)
(19, 19, 47045881, 56)
(31, 31, 819628286980801, 154)
(37, 37, 6582952005840035281, 221)
(43, 43, 73885357344138503765449, 300)
(61, 61, 508858109619679129936596364708525201, 609)
(67, 67, 14915769363385151583217201855136979828889, 736)
(73, 73, 524503804723748275248688345080154231098133441, 875)
(79, 79, 21792365464956367829706954331797367137260238877921, 1026)
[6, 25, 56, 154, 221, 300, 609, 736, 875, 1026]


In [67]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]:
            xx=p-1
            m=p^xx
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,o)
            data=data+[o]
            print(tupl)
print(data) # no OEIS entry

(5, 5, 18)
(7, 7, 40)
(11, 11, 108)
(13, 13, 154)
(17, 17, 270)
(19, 19, 340)
(23, 23, 504)
(29, 29, 810)
(31, 31, 928)
(37, 37, 1330)
(41, 41, 1638)
(43, 43, 1804)
(47, 47, 2160)
(53, 53, 2754)
(59, 59, 3420)
(61, 61, 3658)
(67, 67, 4420)
(71, 71, 4968)
(73, 73, 5254)
(79, 79, 6160)
(83, 83, 6804)
(89, 89, 7830)
[18, 40, 108, 154, 270, 340, 504, 810, 928, 1330, 1638, 1804, 2160, 2754, 3420, 3658, 4420, 4968, 5254, 6160, 6804, 7830]


In [68]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]:
            xx=2*p
            m=p^xx
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,o/2)
            data=data+[o/2]
            print(tupl)
print(data) # A084920		a(n) = (prime(n)-1)*(prime(n)+1). (n = 3, 4, ...)

(5, 5, 24)
(7, 7, 48)
(11, 11, 120)
(13, 13, 168)
(17, 17, 288)
(19, 19, 360)
(23, 23, 528)
(29, 29, 840)
(31, 31, 960)
(37, 37, 1368)
(41, 41, 1680)
(43, 43, 1848)
(47, 47, 2208)
(53, 53, 2808)
(59, 59, 3480)
(61, 61, 3720)
(67, 67, 4488)
(71, 71, 5040)
(73, 73, 5328)
(79, 79, 6240)
(83, 83, 6888)
(89, 89, 7920)
[24, 48, 120, 168, 288, 360, 528, 840, 960, 1368, 1680, 1848, 2208, 2808, 3480, 3720, 4488, 5040, 5328, 6240, 6888, 7920]


In [69]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]:
            xx=3*p
            m=p^xx
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,o)
            data=data+[o]
            print(tupl)
print(data) # no OEIS

(5, 5, 73)
(7, 7, 145)
(11, 11, 361)
(13, 13, 505)
(17, 17, 865)
(19, 19, 1081)
(23, 23, 1585)
(29, 29, 2521)
(31, 31, 2881)
(37, 37, 4105)
(41, 41, 5041)
(43, 43, 5545)
(47, 47, 6625)
(53, 53, 8425)
(59, 59, 10441)
(61, 61, 11161)
(67, 67, 13465)
(71, 71, 15121)
(73, 73, 15985)
(79, 79, 18721)
(83, 83, 20665)
(89, 89, 23761)
[73, 145, 361, 505, 865, 1081, 1585, 2521, 2881, 4105, 5041, 5545, 6625, 8425, 10441, 11161, 13465, 15121, 15985, 18721, 20665, 23761]


In [73]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]:
            xx=p^2+1
            m=p^xx
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,o/4)
            data=data+[o/4]
            print(tupl)
print(data) # no OEIS
dt=difference_table(data)
print(dt) # no OEIS

(5, 5, 32)
(7, 7, 87)
(11, 11, 335)
(13, 13, 552)
(17, 17, 1232)
(19, 19, 1719)
(23, 23, 3047)
(29, 29, 6104)
(31, 31, 7455)
(37, 37, 12672)
(41, 41, 17240)
(43, 43, 19887)
(47, 47, 25967)
(53, 53, 37232)
(59, 59, 51359)
(61, 61, 56760)
(67, 67, 75207)
(71, 71, 89495)
(73, 73, 97272)
(79, 79, 123279)
(83, 83, 142967)
(89, 89, 176264)
[32, 87, 335, 552, 1232, 1719, 3047, 6104, 7455, 12672, 17240, 19887, 25967, 37232, 51359, 56760, 75207, 89495, 97272, 123279, 142967, 176264]
[55, 248, 217, 680, 487, 1328, 3057, 1351, 5217, 4568, 2647, 6080, 11265, 14127, 5401, 18447, 14288, 7777, 26007, 19688, 33297]


In [75]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]:
            xx=p^2-1
            m=p^xx
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,o/2)
            data=data+[o/2]
            print(tupl)
print(data) 

(5, 5, 59)
(7, 7, 167)
(11, 11, 659)
(13, 13, 1091)
(17, 17, 2447)
(19, 19, 3419)
(23, 23, 6071)
(29, 29, 12179)
(31, 31, 14879)
(37, 37, 25307)
(41, 41, 34439)
(43, 43, 39731)
(47, 47, 51887)
(53, 53, 74411)
(59, 59, 102659)
(61, 61, 113459)
(67, 67, 150347)
(71, 71, 178919)
(73, 73, 194471)
(79, 79, 246479)
(83, 83, 285851)
(89, 89, 352439)
[59, 167, 659, 1091, 2447, 3419, 6071, 12179, 14879, 25307, 34439, 39731, 51887, 74411, 102659, 113459, 150347, 178919, 194471, 246479, 285851, 352439]


In [78]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]:
            xx=p^2+p+1
            m=p^xx
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,o)
            data=data+[o]
            print(tupl)
print(data) 
dt=difference_table(data)
print(dt)

(5, 5, 153)
(7, 7, 397)
(11, 11, 1461)
(13, 13, 2377)
(17, 17, 5217)
(19, 19, 7237)
(23, 23, 12717)
(29, 29, 25257)
(31, 31, 30781)
(37, 37, 52057)
(41, 41, 70641)
(43, 43, 81397)
(47, 47, 106077)
(53, 53, 151737)
(59, 59, 208917)
(61, 61, 230761)
(67, 67, 305317)
(71, 71, 363021)
(73, 73, 394417)
(79, 79, 499357)
(83, 83, 578757)
(89, 89, 712977)
[153, 397, 1461, 2377, 5217, 7237, 12717, 25257, 30781, 52057, 70641, 81397, 106077, 151737, 208917, 230761, 305317, 363021, 394417, 499357, 578757, 712977]
[244, 1064, 916, 2840, 2020, 5480, 12540, 5524, 21276, 18584, 10756, 24680, 45660, 57180, 21844, 74556, 57704, 31396, 104940, 79400, 134220]


In [79]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..24]:
    p=prime(n)
    firsts=[]
    for k in [p-1..p-1]: 
        power=s[k][0] 
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..1]:
            xx=p^2+2*p+1
            m=p^xx
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            tupl=(p,power,o)
            data=data+[o]
            print(tupl)
print(data) 
dt=difference_table(data)
print(dt)

(5, 5, 178)
(7, 7, 446)
(11, 11, 1582)
(13, 13, 2546)
(17, 17, 5506)
(19, 19, 7598)
(23, 23, 13246)
(29, 29, 26098)
(31, 31, 31742)
(37, 37, 53426)
(41, 41, 72322)
(43, 43, 83246)
(47, 47, 108286)
(53, 53, 154546)
(59, 59, 212398)
(61, 61, 234482)
(67, 67, 309806)
(71, 71, 368062)
(73, 73, 399746)
(79, 79, 505598)
(83, 83, 585646)
(89, 89, 720898)
[178, 446, 1582, 2546, 5506, 7598, 13246, 26098, 31742, 53426, 72322, 83246, 108286, 154546, 212398, 234482, 309806, 368062, 399746, 505598, 585646, 720898]
[268, 1136, 964, 2960, 2092, 5648, 12852, 5644, 21684, 18896, 10924, 25040, 46260, 57852, 22084, 75324, 58256, 31684, 105852, 80048, 135252]
